In [1]:
import pandas as pd

In [29]:
df = pd.read_csv("./Dataset/Toy_Low/mapping.tsv", sep='\t')
genomes = df['genome_id'].value_counts()
df = df[["#anonymous_gsa_id", "tax_id"]]
df.to_csv(busybee_0_percent_output,index=None,sep='\t')

,#anonymous_gsa_id,tax_id
0,S|S1|C43146,1122939
1,S|S1|C3270,1122939


# Create keanu database

In [30]:
%%bash
python3 ./keanu/make_db.py -names ./Taxonomy/names.dmp -nodes ./Taxonomy/nodes.dmp  -deleted ./Taxonomy/delnodes.dmp -merged ./Taxonomy/merged.dmp  -out_db ./Taxonomy/taxonomy.dat -out_md_db ./Taxonomy/merged_deleted.dat

# Formatting the input

In [ ]:
%%bash
python3 ./keanu/format_input.py -in query.staxids.txt -out sample_name.keanu.txt